In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import scanpy as sc,anndata as ad
import squidpy as sq
import os
import sys
from scipy.sparse import vstack
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,scale
from scipy.spatial import distance_matrix, distance
from sklearn.neighbors import KernelDensity
from scipy.spatial import KDTree
from scipy.spatial.distance import cdist
import networkx as nx
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
from sklearn.neighbors import BallTree
import time
import rdata
from scipy.sparse import csr_matrix,bsr_matrix,coo_matrix,issparse,lil_matrix,diags
from scipy.sparse.linalg import inv
import scipy as sp
from  scipy.ndimage import gaussian_filter
import igraph as ig
import glasbey
import warnings
import cairocffi as cairo
from sklearn.metrics import adjusted_rand_score,make_scorer
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn import preprocessing
import libpysal
from esda.losh import LOSH
from multiprocessing import Pool
from mclustpy import mclustpy
from sklearn.cluster import KMeans
import torch
#from GraphST import GraphST
import SEDR
#from GraphST.utils import clustering
from matplotlib.colors import ListedColormap
from esda import Moran
from libpysal.weights import KNN
from scsampler import scsampler 
from geosketch import gs
from scvalue import SCValue
from tqdm import tqdm
from fbpca import pca
from annoy import AnnoyIndex
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import jensenshannon
from sklearn.metrics import mean_squared_error
from numpy.linalg import norm

In [2]:
from functions import *

In [19]:
def sampling_methods_get_index(adata, fraction=0.1, seed=0, k=20):
    """
    Evaluates different sampling methods for spatial transcriptomics data and computes partial Hausdorff distances.

    Parameters:
    - adata: AnnData object
    - fraction: float, fraction of cells to sample
    - seed: int, random seed for reproducibility
    - k: int, number of principal components for PCA-based geosketching

    Returns:
    - A dictionary mapping method names to sampled indices
    - A pandas DataFrame with computed distances
    """
    start = time.time()
    print("starting eval")
    
    sampled_data = {}
    print("running geosketch on transcripts now")
    # PCA-based geosketching
    U, s, _ = pca(adata.X, k=k)
    X_dimred = U[:, :k] * s[:k]
    N = int(fraction * adata.X.shape[0])
    geo_transcriptomic_index = gs(X_dimred, N, seed=seed, replace=False)
    res = scsampler(adata.obsm['X_pca'], fraction=fraction, random_state=seed, copy=True,random_split = 16)
    scsampler_transcptomic_index = res[1]
    print("running coordinate based sampling now")
    # Coordinate-based sampling
    scsampler_coord_index = scsampler(adata.obsm['spatial'], fraction=fraction, random_state=seed, copy=True,random_split = 16)[1]
    geo_coord_index = gs(adata.obsm['spatial'], N, seed=seed, replace=False)

    return({'geo_transcriptomic':geo_transcriptomic_index,'geo_coords':geo_coord_index,"scsampler_transcriptomic":scsampler_transcptomic_index,
            'scsampler_coods':scsampler_coord_index})


def sampling_methods_get_uniform_and_shuffled_index(adata,fraction = 0.1,seed = 0, k = 20):
    print("starting eval")
    

    uniform_indices = uniform_index(adata,fraction = fraction,seed = seed)
    shuffled_geo = geo_coord_shuffled_index(adata,fraction = fraction, seed = seed)
    shuffled_scsampler = scsampler_coord_shuffled_index(adata,fraction = fraction,seed = seed)

    return({'uniform':uniform_indices,'shuffled_geo':shuffled_geo,'shuffled_scsampler':shuffled_scsampler})
    


def geo_coord_index(adata,fraction =0.1,seed =0): 
    N =int(fraction * adata.X.shape[0])
    geo_coord_index = gs(adata.obsm['spatial'], N, seed=seed, replace=False)
    return geo_coord_index
def geo_transcriptomic_index(adata,fraction = 0.1,seed = 0):
    N =int(fraction * adata.X.shape[0])
    geo_transcriptomic_index = gs(adata.obsm['X_pca'], N, seed=seed, replace=False)
    return geo_transcriptomic_index

def geo_coord_shuffled_index(adata,fraction = 0.1, seed = 0):
    N =int(fraction * adata.X.shape[0])
    coords = adata.obsm['spatial']
    shuffled_coords = shuffle_coordinates(coords,seed = seed)
    geo_coord_shuffled_index = gs(shuffled_coords,N,seed = seed,replace = False)
    return geo_coord_shuffled_index
    
def scsampler_coord_index(adata,fraction = 0.1,seed = 0):
    res = scsampler(adata.obsm['spatial'], fraction=fraction, random_state=seed, copy=True,random_split = 16)
    scsampler_transcptomic_index = res[1]
    return sorted(list(scsampler_transcptomic_index))

def scsampler_transcriptomics_index(adata,fraction = 0.1,seed = 0):
    res = scsampler(adata.obsm['X_pca'], fraction=fraction, random_state=seed, copy=True,random_split = 16)
    return sorted(list(res[1]))


def scsampler_coord_shuffled_index(adata,fraction = 0.1,seed = 0):
    coords = adata.obsm['spatial']
    shuffled_coords = shuffle_coordinates(coords,seed = seed)
    res = scsampler(shuffled_coords,fraction = fraction,random_state = seed,copy = True,random_split = 16)
    return sorted(list(res[1]))

def uniform_index(adata,fraction = 0.1,seed = 0):
    np.random.seed(seed)
    num_cells = adata.n_obs
    sample_size = int(num_cells*fraction)
    sampled_indices = np.random.choice(adata.obs.index,size = sample_size,replace = False)
    return sorted(list(sampled_indices))



def shuffle_coordinates(coordinates,seed = 0):
    """Shuffles pairs of coordinates in a NumPy array.

    Args:
        coordinates (numpy.ndarray): An array of shape (N, 2) 
                                     where N is the number of coordinate pairs.

    Returns:
        numpy.ndarray: A new array with shuffled coordinate pairs.
    """
    rng = np.random.default_rng(seed = seed)
    shuffled_indices = rng.permutation(coordinates.shape[0])
    shuffled_coordinates = coordinates[shuffled_indices]
    return shuffled_coordinates

In [17]:
directory = "/dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/data"
adata = sc.read_h5ad(os.path.join(directory,"region0_0824.h5ad"))
pca_model = PCA(n_components=20, svd_solver='randomized', random_state=2024)
pca_data = pca_model.fit_transform(adata.X.toarray())
adata.obsm['X_pca'] = pca_data

In [20]:
#do geosketch and scsampler on the shuffled coords and real coords. 
# Parent directory
parent_dir = '/dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index'

sub_dirs = {
    'uniform': uniform_index,
    'geo_shuffled': geo_coord_shuffled_index,
    'scsampler_shuffled':scsampler_coord_shuffled_index
}

# Fraction and seeds
fractions = [0.01, 0.02, 0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
seeds = range(10)  

for fraction in fractions:
    # Loop through each subdirectory and corresponding function
    for sub_dir, index_function in sub_dirs.items():
        result_lists = {f'seed_{seed}': [] for seed in seeds}  # Create a dictionary for results
    
        # Run the function with different seeds
        for seed in seeds:
            # You need to define `adata` appropriately (not shown)
            result = index_function(adata, fraction=fraction, seed=seed)
            result_lists[f'seed_{seed}'] = result  # Store result in the dictionary under the appropriate seed key
    
        # Create a DataFrame from the result lists
        df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in result_lists.items()]))
    
        # Save DataFrame to CSV in corresponding subdirectory
        output_file_path = os.path.join(parent_dir, sub_dir, f'index_{fraction}.csv')
        df.to_csv(output_file_path, index=False)
    
        print(f'Results saved in {output_file_path}')

Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/uniform/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/geo_shuffled/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/scsampler_shuffled/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/uniform/index_0.02.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/geo_shuffled/index_0.02.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/scsampler_shuffled/index_0.02.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/uniform/index_0.03

In [33]:
# Parent directory
parent_dir = '/dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index'

# Subdirectories and corresponding functions
sub_dirs = {
    'geo_coords': geo_coord_index,
    'geo_transcriptomic': geo_transcriptomic_index,
    'scsampler_coords': scsampler_coord_index,
    'scsampler_transcriptomic': scsampler_transcriptomics_index
}

# Fraction and seeds
fractions = [0.01, 0.02, 0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
seeds = range(10)  # Generates seeds [0, 1, 2, 3, 4]

for fraction in fractions:
    # Loop through each subdirectory and corresponding function
    for sub_dir, index_function in sub_dirs.items():
        result_lists = {f'seed_{seed}': [] for seed in seeds}  # Create a dictionary for results
    
        # Run the function with different seeds
        for seed in seeds:
            # You need to define `adata` appropriately (not shown)
            result = index_function(adata, fraction=fraction, seed=seed)
            result_lists[f'seed_{seed}'] = result  # Store result in the dictionary under the appropriate seed key
    
        # Create a DataFrame from the result lists
        df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in result_lists.items()]))
    
        # Save DataFrame to CSV in corresponding subdirectory
        output_file_path = os.path.join(parent_dir, sub_dir, f'index_{fraction}.csv')
        df.to_csv(output_file_path, index=False)
    
        print(f'Results saved in {output_file_path}')

Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/geo_coords/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/geo_transcriptomic/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/scsampler_coords/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/scsampler_transcriptomic/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/geo_coords/index_0.02.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/geo_transcriptomic/index_0.02.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/in

/dartfs-hpc/rc/lab/F/FrostH/members/igingerich/envs/miniconda3/envs/r-kernel/lib/python3.12/site-packages/scsampler/uclab.py:46: RuntimeWarning: divide by zero encountered in divide
  d = np.power(1/d, alpha)


Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/index/scsampler_transcriptomic/index_0.99.csv


In [21]:
directory = "/dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/processed_data"
adata = sc.read_h5ad(os.path.join(directory,"xenium_processed.h5ad"))
pca_model = PCA(n_components=20, svd_solver='randomized', random_state=2024)
pca_data = pca_model.fit_transform(adata.X.toarray())
adata.obsm['X_pca'] = pca_data

In [ ]:
parent_dir = '/dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index'

sub_dirs = {
    'uniform': uniform_index,
    'geo_shuffled': geo_coord_shuffled_index,
    'scsampler_shuffled':scsampler_coord_shuffled_index
}

# Fraction and seeds
fractions = [0.01, 0.02, 0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
seeds = range(10)  

for fraction in fractions:
    # Loop through each subdirectory and corresponding function
    for sub_dir, index_function in sub_dirs.items():
        result_lists = {f'seed_{seed}': [] for seed in seeds}  # Create a dictionary for results
    
        # Run the function with different seeds
        for seed in seeds:
            # You need to define `adata` appropriately (not shown)
            result = index_function(adata, fraction=fraction, seed=seed)
            result_lists[f'seed_{seed}'] = result  # Store result in the dictionary under the appropriate seed key
    
        # Create a DataFrame from the result lists
        df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in result_lists.items()]))
    
        # Save DataFrame to CSV in corresponding subdirectory
        output_file_path = os.path.join(parent_dir, sub_dir, f'index_{fraction}.csv')
        df.to_csv(output_file_path, index=False)
    
        print(f'Results saved in {output_file_path}')

Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/uniform/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/geo_shuffled/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/scsampler_shuffled/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/uniform/index_0.02.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/geo_shuffled/index_0.02.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/scsampler_shuffled/index_0.02.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/uniform/index_0.03.csv
Results saved in /dartfs-hpc/rc/lab/F

In [35]:
# Parent directory
parent_dir = '/dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index'

# Subdirectories and corresponding functions
sub_dirs = {
    'geo_coords': geo_coord_index,
    'geo_transcriptomic': geo_transcriptomic_index,
    'scsampler_coords': scsampler_coord_index,
    'scsampler_transcriptomic': scsampler_transcriptomics_index
}

# Fraction and seeds
fractions = [0.01, 0.02, 0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
seeds = range(10)  # Generates seeds [0, 1, 2, 3, 4]

for fraction in fractions:
    # Loop through each subdirectory and corresponding function
    for sub_dir, index_function in sub_dirs.items():
        result_lists = {f'seed_{seed}': [] for seed in seeds}  # Create a dictionary for results
    
        # Run the function with different seeds
        for seed in seeds:
            # You need to define `adata` appropriately (not shown)
            result = index_function(adata, fraction=fraction, seed=seed)
            result_lists[f'seed_{seed}'] = result  # Store result in the dictionary under the appropriate seed key
    
        # Create a DataFrame from the result lists
        df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in result_lists.items()]))
    
        # Save DataFrame to CSV in corresponding subdirectory
        output_file_path = os.path.join(parent_dir, sub_dir, f'index_{fraction}.csv')
        df.to_csv(output_file_path, index=False)
    
        print(f'Results saved in {output_file_path}')

Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/geo_coords/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/geo_transcriptomic/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/scsampler_coords/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/scsampler_transcriptomic/index_0.01.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/geo_coords/index_0.02.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/geo_transcriptomic/index_0.02.csv
Results saved in /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/xenium_brain/sketching_test/index/scsampler_coords/index_0.02.csv
Result